In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-success">
    <h1>Problema: Edificis</h1>
    <p>
        Volem solucionar el puzzle dels edificis. Donada una graella com la que tenim a continuació (esquerra), volem trobar la solució al puzzle (dreta):
        <center><img src="img/sky-intro.png" width='25%'></center>
        Podem pensar que cada una de les caselles és un solar on hi podem posar un edifici de X plantes. En aquest cas un edifici de 1,2,3 o 4 plantes ja que el tauler és de 4x4.<br>
        Els números que trobem a fora de la graella ens indiquen quants edificis són visibles des d'aquesta posició. Un edifici d'alçada 4 tapa qualsevol edifici que hi ha al darrere com podeu veure en la imatge següent:
        <center><img src="img/sky-rule-3-1.png" width='25%'></center>
        A més, en cada fila i columna de la graella només pot haver-hi un sol edifici de cada alçada, és a dir, no es poden repetir.
        <center><img src="img/skyscrapers.png" width='25%'></center>
        L'objectiu és doncs, donada una graella com la inicial, trobar la configuració d'edificis que satisfà les condicions. Implementeu un algorisme usant backtracking que <b>trobi una solució utilitzant el mínim nombre de crides recursives i minimitzant el temps total d'execució</b>. Per fer-ho haureu de tenir en comptes diferents factors com ara:
        <ol>
            <li> Quin número començareu provant?
            <li> Quina casella serà la primera que omplireu?
        </ol>
    </p>
    
</div>

La funció **format_sky** us ajudarà a visualitzar el tauler del problema.

In [1]:
import numpy as np
from IPython.display import clear_output
import time

def format_sky(board, top, bottom, left, right):
    """
    Funció auxiliar per mostrar la matriu del Problema 1 en forma de graella.
    """
    _str = ""

    # Files
    for v in top:
        _str+="   "+str(v)
    _str = "  " + _str + "\n"

    for idx, i in enumerate(board):
        _str+= "   +"+ ("-"*((board.shape[1]*4)-1)) +"+\n"
        _str+= f" {left[idx]} | "

        # Columnes
        for j in i:
            if j!=0:
                _str+= str(j)+" | "
            else:
                 _str+= "  | "

        _str+= f"{right[idx]} \n"
    _str+= "   +"+ ("-"*((board.shape[1]*4)-1))+"+\n"

    _bot = ""
    for v in bottom:
        _bot+="   "+str(v)
    _str += "  " + _bot

    return _str.replace('0',' ')

Veieu un exemple de com utilitzar aquesta funció:


In [3]:
def pop_method(lista, n , condicio, posicio):
    for num_borrar in range(n, n-condicio+posicio, -1):
        print(num_borrar)
        lista.remove(num_borrar)
    if condicio == 2 and posicio == 2:
        lista.remove(5-1)
    print(lista)

In [ ]:
pop_method([1,2,3,4,5], 4, 2, 2)

In [ ]:
# Amb aquesta funció es mostren totes les caselles del tauler
# Les caselles que contenen un zero es mostren buides

# Creem unes restriccions i un tauler
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

print('Tauler:')
print(format_sky(grid, top, bottom, left, right))

La funció **delayed_print** us permet bloquejar temporalment un 'print' per si voleu veure les accions que fa a cada pas el vostre algorisme.<br>
No heu d'utilitzar-la a l'entrega ja que el programa tardarà molt més del necessari, utilitzeu-la només mentre programeu la solució.

In [21]:
def delayed_print(grid, top, bottom, left, right, sleep_time=0.02):
    """
    Funció auxiliar que bloqueja temporalment la funció 'print' per veure l'evolució de l'algorisme.
    """
    print(format_sky(grid, top, bottom, left, right))
    clear_output(wait=True)
    time.sleep(sleep_time)

In [ ]:
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

for i in range(0, len(left)):
    for j in range(0, len(top)):
        grid[i][j] = np.random.randint(1,4)
        delayed_print(grid, top, bottom, left, right, sleep_time=0.25)

In [ ]:
#SEMPRE TENIM EN COMPTE EN LES EXPLICACIONS QUE n ES IGUAL A LA LONGITUD DEL TAULER
#                                               k ES IGUAL A LA DISTANCIA DESDE LA CONDICIO ON MIREM FINS A LA CASELLA ON VOLEM POSAR UN EDIFICI
#                                               i ES IGUAL A LA CONDICIO 
#                                               x ES IGUAL A ELS NUMEROS QUE PODEN ANAR A LA CASELLA

from collections import Counter

# A aquest metode resolem 3 problemes:
# 1. Si hi han n llistes de n possibilitats, idèntiques entre si, només aquestes caselles podem tenir els numeros continguts, es a dir, hem d'esborrar aquestes possibilitats de les altres llistes.
# 2. Si tenim algun numero nomes a una sola llista de les caselles d'una mateix fila o columna, vol dir que aquest numero nomes pot anar a dita casella.
# 3. Si tenim un edifici d'altura n a la distancia i d'una condicio (per exemple a un tauler 5x5, si la condicio de top a la esquerra del tot es un 3, si trobem un 5 a la fila 3 columna 1)
#    En aquestes condicions --> x >= k

def procesar_listas(lista_principal, left_top, right_bottom):
    n = len(lista_principal)

    # Crear una llista temporal per emmagatzemar les subllistes com a tuples
    subllistes_tuples = []

    # Omplir la llista subllistes_tuples amb les subllistes convertides a tuples
    for subllista in lista_principal:
        subllistes_tuples.append(tuple(subllista))

    # Comptar la frequencia de les subllistes basant-se en el seu contingut
    subllistes_comptador = Counter(subllistes_tuples)

    # Crear un conjunt per emmagatzemar els numeros a eliminar
    numeros_a_eliminar = set()
    subllistes_relevants = set()  # Guardem les subllistes rellevants

    # Identificar subllistes rellevants i els numeros a eliminar
    for subllista, comptador in subllistes_comptador.items():
        longitud = len(subllista)
        # Verificar si hi ha prou subllistes identiques per ser rellevants
        if comptador >= longitud:
            # Marcar aquesta subllista com a rellevant
            subllistes_relevants.add(subllista)
            # Afegir els numeros d'aquesta subllista al conjunt d'eliminacio
            for num in subllista:
                numeros_a_eliminar.add(num)

    # Crear la llista resultant
    lista_resultante = []
    for subllista in lista_principal:
        # Si la subllista es rellevant, no eliminem els seus elements
        if tuple(subllista) in subllistes_relevants:
            lista_resultante.append(subllista)
        else:
            # Filtrar els numeros a eliminar nomes de les subllistes no rellevants
            nova_subllista = []
            for num in subllista:
                if num not in numeros_a_eliminar:
                    nova_subllista.append(num)
            lista_resultante.append(nova_subllista)

    # Llista on emmagatzemem a la posicio del numero quantes vegades es troba aquest a totes les llistes
    estan = [0] * n

    # Guardem en quina casella es troba el numero que nomes esta un cop
    donde_esta = -1

    for possibilitats in lista_resultante:
            for numero in possibilitats:
                estan[numero - 1] += 1
                # La condicio de si len(possibilitats) != 1 es perque ho fem a un altre metode
                if estan[numero - 1] == 1 and len(possibilitats) != 1:
                    donde_esta = lista_resultante.index(possibilitats)
                    num = numero

    if 1 in estan:
        for i in range(len(estan)):
            if estan[i] == 1 and (i + 1) in lista_resultante[donde_esta]:
                lista_resultante[donde_esta] = [i + 1]


    # if k de n == i --> x >= k
    for i in range(n):
        if lista_resultante[i] == [n]:
            if (i + 1) == left_top:
                for j in range(i):
                    for k in range(1, j + 1):
                        if k in lista_resultante[j]:
                            lista_resultante[j].remove(k)
            if(n - i) == right_bottom:
                for j in range(n - 1, i, -1):
                    for k in range(1, n - j):
                        if k in lista_resultante[j]:
                            lista_resultante[j].remove(k)
            break   
    
    return lista_resultante

# Aquesta funcio posa n al costat de les condicions 1
# i posa 1, 2, ..., n al costat de les condicions n
def uns_i_maxims(grid, top, bottom, left, right):

    if 1 in top or len(grid) in top:
        for i in range(len(grid)):
            if top[i] == 1:
                grid[0][i] = len(grid)
            elif top[i] == len(grid):
                for j in range(len(grid[i])):
                    grid[j][i] = j + 1

    if 1 in bottom or len(grid) in bottom:
        for i in range(len(grid)):
            if bottom[i] == 1:
                grid[len(grid)-1][i] = len(grid)
            elif bottom[i] == len(grid):
                grid[len(grid[i]) - 1][i] = 1
                for j in range(len(grid[i]) - 2, -1, -1):
                    grid[j][i] = grid[j+1][i] + 1

    if 1 in left or len(grid) in left:
        for i in range(len(grid)):
            if left[i] == 1:
                grid[i][0] = len(grid)
            elif left[i] == len(grid):
                for j in range(len(grid[i])):
                    grid[i][j] = j + 1
    
    if 1 in right or len(grid) in right:
        for i in range(len(grid)):
            if right[i] == 1:
                grid[i][len(grid)-1] = len(grid)
            elif right[i] == len(grid):
                grid[i][len(grid[i]) - 1] = 1
                for j in range(len(grid[i]) - 2, -1, -1):
                    grid[i][j] = grid[i][j+1] + 1

# A aquesta funcio li arriba una casella de la possibilities_grid, i aplica les seguents formules: 
# 1. x <= n - i + k
# 2. if i == 2 and k == 2 --> x != n-1

def pop_method(lista, n , condicio, posicio):

    for num_borrar in range(n, n-condicio+posicio, -1):
        if num_borrar in lista:
            lista.remove(num_borrar)

    if condicio == 2 and posicio == 2 and (n-1) in lista:
        lista.remove(n-1)

    return lista

# Aquesta funcio ompleix la matriu possibilities_grid amb les llistes de possibilitats de numeros que poden anar a la casella (x,y) de la grid

def fill_possibilities_grid(possibilities_grid, top, bottom, left, right):

    total_list = []
    n = len(possibilities_grid)

    # Fem una llista amb [n, n-1, ..., 1] --> Ho fem de major a menor
    for i in range(n, 0, -1):
        total_list.append(i)

    # Omplim la matriu possibilities_grid amb els numeros ja fixes de la grid i amb les llistes abans fetes
    for i in range(n):
        for j in range(n):
            if possibilities_grid[i][j] == 0:
                possibilities_grid[i][j] = total_list.copy()
            else:
                possibilities_grid[i][j] = [possibilities_grid[i][j]]

    condicions_tractades = {0, 1, n}
    
    # Enviem totes les caselles de la grid amb les condicions que li afecten i no s'han tractat ja, al pop_method
    for i in range(n): 
        if top[i] not in condicions_tractades:
            for j in range(n):
                possibilities_grid[j][i] = pop_method(possibilities_grid[j][i], n, top[i], j+1)
    
        if bottom[i] not in condicions_tractades:
            for j in range(n):
                possibilities_grid[j][i] = pop_method(possibilities_grid[j][i], n, bottom[i], n-j)

        if left[i] not in condicions_tractades:
            for j in range(n):
                possibilities_grid[i][j] = pop_method(possibilities_grid[i][j], n, left[i], j+1)
    
        if right[i] not in condicions_tractades:
            for j in range(n):
                possibilities_grid[i][j] = pop_method(possibilities_grid[i][j], n, right[i], n-j)
    
    # Mentre s'hagi fet algun canvi en la possibilities_grid, repetim el procés perquè aquest canvi pot encadenar en mes canvis
    while True:

        anterior_grid = possibilities_grid.copy()

        for i in range(n):

            possibilities_grid[i] = procesar_listas(possibilities_grid[i], left[i], right[i])

            lista_columna = []

            for j in range(n):
                lista_columna.append(possibilities_grid[j][i])

            lista_resultante = procesar_listas(lista_columna, top[i], bottom[i])

            for j in range(n):
                possibilities_grid[j][i] = lista_resultante[j]

        if anterior_grid == possibilities_grid:
            break

            


# Comprovem que ficant el nombre indicat a satisfies no s'incompleix la condicio de top
def comprova_top(grid, y, top):

    # Si no hi ha condicio es compleix
    if top[y] == 0:
        return True

    visibles = 0
    altura_max = 0

    # Si a la columna encara hi ha un 0 es pot complir per tant retornem True
    for i in range(len(grid)):
        if grid[i][y] == 0:
            return True

    # Calculem que la condicio es compleix
    for i in range(len(grid)):
        if grid[i][y] > altura_max:
            altura_max = grid[i][y]
            visibles += 1
            if visibles > top[y]:
                return False
            
    if visibles != top[y]:
        return False

    return True

# Els altres comprova son iguals que la de top
def comprova_bottom(grid, y, bottom):
    if bottom[y] == 0:
        return True

    visibles = 0
    altura_max = 0

    for i in range(len(grid)):
        if grid[i][y] == 0:
            return True

    for i in range(len(grid) - 1, -1, -1):
        
        if grid[i][y] > altura_max:
            altura_max = grid[i][y]
            visibles += 1
            if visibles > bottom[y]:
                return False
            
    if visibles != bottom[y]:
        return False

    return True

def comprova_left(grid, x, left):

    if left[x] == 0 or np.any(grid[x] == 0):
        return True

    visibles = 0
    altura_max = 0

    for j in range(len(grid)):
        if grid[x][j] > altura_max:
            altura_max = grid[x][j]
            visibles += 1
            if visibles > left[x]:
                return False
            
    if visibles != left[x]:
        return False

    return True

def comprova_right(grid, x, right):
    if right[x] == 0 or np.any(grid[x] == 0):
        return True

    visibles = 0
    altura_max = 0

    for j in range(len(grid) - 1, -1, -1):
        if grid[x][j] > altura_max:
            altura_max = grid[x][j]
            visibles += 1
            if visibles > right[x]:
                return False
    
    if visibles != right[x]:
        return False

    return True


def satisfies(grid, x, y, num, top, bottom, left, right):
    """
    Donat un tauler, un punt amb coordenades (x,y) i un nombre 'num', comprova si és possible assignar-lo a la posició (x,y)
    donades les restriccions del problema.

    Params
    ------
    :grid: Una matriu de numpy
    :x,y: Un punt de la matriu on volem posar el nombre 'num'
    :num: Un nombre enter
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.

    Returns
    -------
    :b: Un boleà True/False depenent de si el nombre 'num' pot ser col·locat a la casella (x,y) o no.
    """

    # Si ja es troba el numero que volem posar a la fila o columna de la casella i no es a la mateixa casella, retornem False
    for i in range(len(grid)):
        if (i != y and grid[x][i] == num) or (i != x and grid[i][y] == num):
            return False

    grid_mod = np.copy(grid)

    grid_mod[x][y] = num

    return (comprova_bottom(grid_mod, y, bottom) and comprova_left(grid_mod, x, left) and comprova_right(grid_mod, x, right) and comprova_top(grid_mod, y, top))



# A la funcio backtracking cridem a la funcio backtracking_aux amb True per tal de que faci el codi que nomes s'ha de fer a la primera iteracio.

def skyscrapper_backtracking(grid, top, bottom, left, right):
    """
    Funció que implementa l'algorisme de backtracking

    Params
    ------
    :grid: Una matriu de tipus numpy
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.

    Returns
    -------
    :grid: La matriu plena amb la solució correcta o bé False en cas que no tingui solució.
    """


    def backtracking_aux(grid, top, bottom, left, right, primer, fila, columna):

        # NO esborreu aquestes dues línies, serveien per saber quants cops es crida aquesta funció
        # ----------------------------------------------------------------------------------------
        global num_recursive_calls
        num_recursive_calls += 1
        # ----------------------------------------------------------------------------------------

        n = len(grid)
       
        if primer:
            uns_i_maxims(grid, top, bottom, left, right)
            global possibilities_grid
            possibilities_grid = grid.copy().tolist()
            fill_possibilities_grid(possibilities_grid, top, bottom, left, right)

            # Per cada posicio de la possibilities_grid que nomes tingui un nombre, si el nombre es pot posar a la casella, ho posem. Sino retornem False.
            for i in range(n):
                for j in range(n):
                    if len(possibilities_grid[i][j]) == 1:
                        if satisfies(grid, i, j, possibilities_grid[i][j][0], top, bottom, left, right):
                            grid[i][j] = possibilities_grid[i][j][0]
                        else:
                            return False
                        
        # Si esta plena i no es la primera iteracio, retornem true. Si es la primera iteracio, retornem la solucio.
        if not np.any(grid == 0):
            return True if num_recursive_calls > 1 else grid

        first_round = True

        for x in range(fila, n):

            # Si es la primera iteracio de la trucada començarem les columnes per la indicada, sino per 0.
            if first_round:
                comienza = columna
                first_round = False
            else:
                comienza = 0

            for y in range(comienza, n):
                if len(possibilities_grid[x][y]) != 1:
                    for num in possibilities_grid[x][y]:

                        if satisfies(grid, x, y, num, top, bottom, left, right):

                            grid[x][y] = num

                            # Si ens trobem a la ultima columna de la fila, la seguent casella es la de la seguent fila primera columna i sino es la de la mateixa fila seguent columna
                            if y == n-1:
                                result = backtracking_aux(grid, top, bottom, left, right, False, x + 1, 0)
                            else:
                                result = backtracking_aux(grid, top, bottom, left, right, False, x, y + 1)

                            # Si es grid o True retornem grid, sino hem de treure el numero posat perque es incorrecte
                            if isinstance(result, np.ndarray) or result is True:
                                return grid
                            
                            grid[x][y] = 0

                    # Si cap numero compleix el satisfies hem de retornar enrere amb False
                    return False

        return False

    return backtracking_aux(grid, top, bottom, left, right, True, 0 , 0)

def skyscrapper(top, bottom, left, right):
    """
    Funció principal del problema. Rep quatre llistes corresponents als nombres que hi ha fora del tauler.

    Params
    ------
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    """

    # Inicialitzem una matriu de zeros
    grid = np.zeros((len(left), len(top)), dtype='int')

    # Cridem a la funció que soluciona el problema mitjançant backtracking
    sol = skyscrapper_backtracking(grid, top, bottom, left, right)

    # Mostrem el resultat en el cas que trobem una solució o mostrem un error en cas contrari.
    if sol is False:
        print('No solution found')
    else:
        
        print(format_sky(np.array(sol), top, bottom, left, right))

In [ ]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([0,0,0],
            [3,0,0],
            [0,0,0],
            [0,0,1])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

In [ ]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([1,3,2,4,2],
            [3,3,3,1,2],
            [1,3,2,4,2],
            [3,3,4,1,2])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')